## Trees: Is this a Binary Tree?
For the purposes of this challenge, we define a binary search tree to be a binary tree with the following properties:

    The data value of every node in a node's left subtree is less than the data value of that node.
    The data value of every node in a node's right subtree is greater than the data value of that node.
    The data value of every node is distinct.

For example, the image on the left below is a valid BST. The one on the right fails on several counts:
- All of the numbers on the right branch from the root are not larger than the root.
- All of the numbers on the right branch from node 5 are not larger than 5.
- All of the numbers on the left branch from node 5 are not smaller than 5.
- The data value 1 is repeated. 

Given the root node of a binary tree, determine if it is a binary search tree.

Function Description Complete the function checkBST in the editor below. It must return a boolean denoting whether or not the binary tree is a binary search tree.

checkBST has the following parameter(s):

    root: a reference to the root node of a tree to test

## Input
The Input from Hackerrank here is non-trivial for a change, as it is not further explained. On Hackerrank, the Tree structure is given to us as input to our function in a Black Box.

In [31]:
from io import StringIO
example_input_valid = lambda: StringIO('''2
1 2 3 4 5 6 7''')
example_input_valid_12 = lambda: StringIO('''3
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15''')
example_input_invalid = lambda: StringIO('''2
1 2 4 3 5 6 7''')

In [3]:
class Node:
    def __init__(self, data, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right
        
    def __repr__(self, depth=0):
        buf = ['{}{}'.format('\t' * depth, self.data)]
        if self.left is not None:
            buf.append(self.left.__repr__(depth=depth+1))
        if self.right is not None:
            buf.append(self.right.__repr__(depth=depth+1))
        return '\n'.join(buf)
        

In [44]:
def make_perfect_btree_node(heap, lower, higher, depth):
    target_idx = int((higher + lower)/ 2)
    if depth == 0:
        return None
    return Node(int(heap[target_idx]),
               make_perfect_btree_node(heap, lower, target_idx, depth - 1),
               make_perfect_btree_node(heap, target_idx, higher, depth - 1))

def construct_perfect_btree_from_input(stream):
    depth = int(stream.readline())
    heap = stream.readline().split()
    return make_perfect_btree_node(heap, 0, len(heap), depth + 1)

In [45]:
construct_perfect_btree_from_input(example_input_valid())

4
	2
		1
		3
	6
		5
		7

In [46]:
construct_perfect_btree_from_input(example_input_valid_12())

8
	4
		2
			1
			3
		6
			5
			7
	12
		10
			9
			11
		14
			13
			15

## Solution

### Naively following the Definition
A simple way to approach this is to simply follow the Definition written in the Challenge description. It can be implemented recursively as follows

In [47]:
def get_vals(root, arr = []):
  if root is None:
    return []
  arr.append(root.data)
  get_vals(root.left, arr)
  get_vals(root.right, arr)
  return arr
    

def __check_bst(root, depth = 0):
  if root is None:
    return True
  left = get_vals(root.left, [])
  rite = get_vals(root.right, [])
  if all(map(lambda x: root.data > x, left)) and all(map(lambda x: root.data < x, rite)):
    return all([__check_bst(root.left, depth = depth + 1), __check_bst(root.right, depth = depth + 1)])
  else:
    return False
  
def has_duplicates(root):
  values = get_vals(root)
  # set() only contains unique elements
  return len(values) != len(set(values))


def checkBST(root):
  # Are we given only one single Node? This is a valid BST.
  if root.left is None and root.right is None:
    return True
  # Are there any duplicates in the Tree? This is disallowed
  # according to the challenge description.
  if has_duplicates(root):
    return False

  # Otherwise, recursively check if the left side of the tree
  # is smaller than the current node, and the right side bigger.
  return __check_bst(root)

In [48]:
checkBST(construct_perfect_btree_from_input(example_input_valid()))

True

In [49]:
checkBST(construct_perfect_btree_from_input(example_input_invalid()))

False

### Using upper and lower Bounds
We can refrain from traversing the tree's children in order to get the elements in each recursion. We're actually only interested in the upper and lower bounds the node value must be confined in. We start with -Inf and +Inf as the Bounds, and adjust the upper bound to be the Node value if we travel to the left, or the lower bound to be the Node value if we travel to the right

In [57]:
def __check_bst_bounds(root, lower, upper):
    if root is None:
        return True
    if root.data > lower and root.data < upper:
        return __check_bst_bounds(root.left, lower, root.data) and __check_bst_bounds(root.right, root.data, upper)
    else: return False

def checkBST_bounds(root):
    return __check_bst_bounds(root, float('-inf'), float('+inf'))
    

In [58]:
checkBST_bounds(construct_perfect_btree_from_input(example_input_valid()))

True

In [59]:
checkBST_bounds(construct_perfect_btree_from_input(example_input_invalid()))

False

## Using the Infix Property
Here's another way to tackle this: valid BST's have the property that they print a sorted list if they are printed in infix notation. We can use this in order to check if we have a valid BST.

In [66]:
def __infix(node, acc):
    if node is None:
        return
    __infix(node.left, acc)
    acc.append(node.data)
    __infix(node.right, acc)
    return acc

def checkBST_infix(root):
    infix = __infix(root, [])
    return len(infix) == len(set(infix)) and sorted(infix) == infix
    

In [67]:
checkBST_infix(construct_perfect_btree_from_input(example_input_valid()))

True

In [68]:
checkBST_infix(construct_perfect_btree_from_input(example_input_invalid()))

False